# Clustering Crypto

In [2]:
# Initial imports
import pandas as pd
import hvplot.pandas
from pathlib import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [3]:
# Load the crypto_data.csv dataset.
file_path = "crypto_data.csv"
df_crypto = pd.read_csv(file_path)
df_crypto.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [76]:
# Keep all the cryptocurrencies that are being traded.
df_crypto_true = df_crypto[df_crypto["IsTrading"] == True].set_index("Unnamed: 0")
df_crypto_true.index.name = None
df_crypto_true.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [81]:
# Remove the "IsTrading" column. 
next_df_crypto = df_crypto_true.drop(["IsTrading"], axis=1)
print(next_df_crypto.shape)
next_df_crypto.head(10)

(1144, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000


In [80]:
# Remove rows that have at least 1 null value.
another_df_crypto = next_df_crypto.dropna()
print(another_df_crypto.shape)
another_df_crypto.head(10)

(685, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000


In [83]:
# Keep the rows where coins are mined.
df_crypto_mined = another_df_crypto[another_df_crypto.TotalCoinsMined > 0]
print(df_crypto_mined.shape)
df_crypto_mined.head(10)

(532, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000
ZEC,ZCash,Equihash,PoW,7.383056e+06,21000000


In [85]:
# Create a new DataFrame that holds only the cryptocurrencies names.
df_crypto_name = df_crypto_mined[["CoinName"]]
print(df_crypto_name.shape)
df_crypto_name.head()

(532, 1)


,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [88]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
new_df_crypto = df_crypto_mined.drop(["CoinName"], axis=1)
print(new_df_crypto.shape)
new_df_crypto.head(10)

(532, 4)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
LTC,Scrypt,PoW,6.303924e+07,84000000
DASH,X11,PoW/PoS,9.031294e+06,22000000
XMR,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethash,PoW,1.133597e+08,210000000
ZEC,Equihash,PoW,7.383056e+06,21000000


In [89]:
# Use get_dummies() to create variables for text features.
df_crypto_encoded = pd.get_dummies(new_df_crypto, columns=["Algorithm", "ProofType"])
df_crypto_encoded[['TotalCoinsMined', 'TotalCoinSupply']] = new_df_crypto[['TotalCoinsMined', 'TotalCoinSupply']]
cols = list(df_crypto_encoded.columns[-2:]) + list(df_crypto_encoded.columns[:-2])
df_crypto_encoded = df_crypto_encoded.reindex(columns=cols)

print(df_crypto_encoded.shape)
df_crypto_encoded.head(10)

(532, 98)


,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,...,ProofType_PoW + Hive,ProofType_PoW and PoS,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS
42,0,0,4.199995e+01,42,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
404,0,0,1.055185e+09,532000000,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1337,0,0,2.927942e+10,314159265359,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
BTC,0,0,1.792718e+07,21000000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,0,0,1.076842e+08,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LTC,0,0,6.303924e+07,84000000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DASH,0,0,9.031294e+06,22000000,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
XMR,0,0,1.720114e+07,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETC,0,0,1.133597e+08,210000000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEC,0,0,7.383056e+06,21000000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [90]:
# Standardize the data with StandardScaler().
df_crypto_scaled = StandardScaler().fit_transform(df_crypto_encoded)
print(df_crypto_scaled[0:5])

[[-0.0433963  -0.0433963  -0.11710817 -0.1528703  -0.0433963  -0.0433963
  -0.0433963  -0.06142951 -0.07530656 -0.0433963  -0.06142951 -0.06142951
  -0.0433963  -0.0433963  -0.19245009 -0.06142951 -0.09740465 -0.0433963
  -0.11547005 -0.07530656 -0.0433963  -0.0433963  -0.15191091 -0.0433963
  -0.13118084 -0.0433963  -0.0433963  -0.08703883 -0.0433963  -0.0433963
  -0.0433963  -0.0433963  -0.06142951 -0.0433963  -0.08703883 -0.08703883
  -0.08703883 -0.0433963  -0.13118084 -0.13840913 -0.13840913 -0.0433963
  -0.06142951 -0.0433963  -0.07530656 -0.18168574 -0.0433963  -0.0433963
  -0.0433963  -0.07530656 -0.15826614 -0.31491833 -0.0433963  -0.08703883
  -0.07530656 -0.06142951  1.38675049 -0.0433963  -0.0433963  -0.06142951
  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.0433963  -0.0433963  -0.39879994 -0.0433963  -0.18168574 -0.0433963
  -0.08703883 -0.08703883 -0.10680283 -0.0433963  -0.13118084 -0.0433963
  -0.0433963  -0.0433963  -0.0433963  -0.075306

### Deliverable 2: Reducing Data Dimensions Using PCA

In [91]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(df_crypto_scaled)
print(crypto_pca)

[[-0.33952037  0.86859871 -0.51755353]
 [-0.32285685  0.86886333 -0.51811803]
 [ 2.30686315  1.64852149 -0.6983328 ]
 ...
 [ 0.32806128 -2.1672185   0.39842944]
 [-0.1533477  -2.0189153   0.39141771]
 [-0.29399547  0.69236165 -0.26139867]]


In [92]:
# Create a DataFrame with the three principal components.
df_crypto_pca = pd.DataFrame(data=crypto_pca, index=df_crypto_encoded.index, columns=["PC 1", "PC 2", "PC 3"])

df_crypto_pca.head(10)

,PC 1,PC 2,PC 3
42,-0.339520,0.868599,-0.517554
404,-0.322857,0.868863,-0.518118
1337,2.306863,1.648521,-0.698333
BTC,-0.136052,-1.147186,0.126829
ETH,-0.148551,-1.981214,0.432884
LTC,-0.167365,-1.191466,0.029317
DASH,-0.399303,1.186993,-0.411906
XMR,-0.144509,-2.104167,0.421445
ETC,-0.146993,-1.981296,0.432850
ZEC,-0.153347,-2.018915,0.391418


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [93]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df_crypto_pca)
    inertia.append(km.inertia_)

elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

C:\Users\Iris\anaconda3\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:1037: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [94]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(df_crypto_pca)

# Predict clusters
predictions = model.predict(df_crypto_pca)
print(predictions)

[0 0 0 3 3 3 0 3 3 3 0 3 0 0 3 0 3 3 0 0 3 3 3 3 3 0 3 3 3 0 3 0 3 3 0 0 3
 3 3 3 3 3 0 0 3 3 3 3 3 0 0 3 0 3 3 3 3 0 3 3 0 3 0 0 0 3 3 3 0 0 0 0 0 3
 3 3 0 0 3 0 3 0 0 3 3 3 3 0 0 3 3 3 3 0 0 3 0 0 3 3 0 0 3 0 0 3 0 3 0 3 0
 3 0 0 3 3 0 3 3 3 0 3 3 3 3 3 0 0 3 3 3 0 3 0 3 3 0 3 0 3 0 0 3 3 0 3 3 0
 0 3 0 3 0 0 0 3 3 3 3 0 0 0 0 0 3 3 0 0 0 0 0 3 0 0 0 0 0 3 0 3 0 0 3 0 3
 0 0 3 0 3 0 3 0 3 0 0 0 0 3 0 0 0 0 0 3 3 0 0 3 3 0 0 0 0 0 3 0 0 0 0 0 0
 0 0 3 0 0 0 0 0 0 3 3 3 0 0 0 0 3 0 3 0 0 3 0 3 3 0 3 3 0 3 0 0 0 3 0 0 3
 0 0 0 0 0 0 0 3 0 3 0 0 0 0 3 0 3 0 3 3 3 3 0 3 0 0 3 0 3 3 3 0 3 0 3 3 3
 0 3 0 3 0 0 1 3 0 3 3 3 3 3 0 0 3 0 0 0 3 0 3 0 3 0 3 0 0 0 0 3 0 0 3 0 0
 0 3 3 3 3 0 0 0 0 3 0 3 3 3 0 0 3 3 0 0 3 0 3 3 3 0 3 3 0 0 0 3 3 3 0 0 0
 3 3 0 3 3 3 3 0 0 0 3 3 3 0 1 0 0 0 0 3 3 3 3 0 0 0 3 0 3 0 0 0 0 3 0 0 3
 0 0 3 3 0 3 0 3 3 3 3 0 0 3 0 3 0 0 0 0 0 0 3 3 3 0 0 0 0 0 0 3 0 3 3 3 3
 0 0 0 0 3 0 0 3 0 0 3 1 3 0 3 3 0 0 3 0 3 3 3 3 3 0 3 0 3 0 0 3 0 0 0 0 0
 3 3 3 0 0 0 3 0 3 0 3 0 

In [95]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.merge(new_df_crypto, df_crypto_pca, left_index=True, right_index=True)


#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df["CoinName"] = df_crypto_mined["CoinName"]

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.339520,0.868599,-0.517554,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.322857,0.868863,-0.518118,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.306863,1.648521,-0.698333,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.136052,-1.147186,0.126829,Bitcoin,3
ETH,Ethash,PoW,1.076842e+08,0,-0.148551,-1.981214,0.432884,Ethereum,3
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.167365,-1.191466,0.029317,Litecoin,3
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.399303,1.186993,-0.411906,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.144509,-2.104167,0.421445,Monero,3
ETC,Ethash,PoW,1.133597e+08,210000000,-0.146993,-1.981296,0.432850,Ethereum Classic,3
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.153347,-2.018915,0.391418,ZCash,3


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [97]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="Class",
    symbol="Class",
    hover_name="CoinName",
    hover_data=["Algorithm"],
    width=800
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [98]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class'], sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [99]:
# Print the total number of tradable cryptocurrencies.
tradeable_crypto_count = (clustered_df["TotalCoinSupply"] != 0).sum()
print("There are", tradeable_crypto_count, "tradeable cryptocurrencies")

There are 532 tradeable cryptocurrencies


In [100]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
scaler = MinMaxScaler().fit(clustered_df[['TotalCoinSupply', 'TotalCoinsMined']])
clustered_df[['TotalCoinSupply_scaled', 'TotalCoinsMined_scaled']] = scaler.transform(clustered_df[['TotalCoinSupply', 'TotalCoinsMined']])

print(clustered_df[['TotalCoinSupply_scaled', 'TotalCoinsMined_scaled']].values)

[[4.20000000e-11 0.00000000e+00]
 [5.32000000e-04 1.06585544e-03]
 [3.14159265e-01 2.95755135e-02]
 ...
 [1.40022261e-03 9.90135079e-04]
 [2.10000000e-05 7.37028150e-06]
 [1.00000000e-06 1.29582282e-07]]


In [101]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
scaled_df = pd.DataFrame(
    data=scaler.transform(clustered_df[['TotalCoinSupply', 'TotalCoinsMined']]), 
    columns=['TotalCoinSupply_scaled', 'TotalCoinsMined_scaled'], 
    index=clustered_df.index
)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
scaled_df['CoinName'] = clustered_df['CoinName']

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
scaled_df['Class'] = clustered_df['Class']

scaled_df.head(10)

,TotalCoinSupply_scaled,TotalCoinsMined_scaled,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141593e-01,0.029576,EliteCoin,0
BTC,2.100000e-05,0.000018,Bitcoin,3
ETH,0.000000e+00,0.000109,Ethereum,3
LTC,8.400000e-05,0.000064,Litecoin,3
DASH,2.200000e-05,0.000009,Dash,0
XMR,0.000000e+00,0.000017,Monero,3
ETC,2.100000e-04,0.000115,Ethereum Classic,3
ZEC,2.100000e-05,0.000007,ZCash,3


In [102]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".

scaled_df.hvplot.scatter(x="TotalCoinsMined_scaled", y="TotalCoinSupply_scaled", by="Class", hover_cols=["CoinName"])

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined_scaled]   (TotalCoinSupply_scaled,CoinName)